# Setup Environment

In [ ]:
!sudo apt update && sudo apt upgrade -y && sudo apt install protobuf-compiler python3-dev python3-pip python-is-python3 -y
!pip install --upgrade tensorflow-gpu numpy

In [ ]:
import os

if not os.path.isdir("Tensorflow/models"):
    !cd Tensorflow && git clone https://github.com/tensorflow/models.git
    
!cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install .

In [ ]:
WORKSPACE_PATH = 'Tensorflow/workspace'
SCRIPTS_PATH = 'Tensorflow/scripts'
APIMODEL_PATH = 'Tensorflow/models'
ANNOTATION_PATH = f'{WORKSPACE_PATH}/annotations'
IMAGE_PATH = f'{WORKSPACE_PATH}/images'
MODEL_PATH = f'{WORKSPACE_PATH}/models'
PRETRAINED_MODEL_PATH = f'{WORKSPACE_PATH}/pre-trained-models'
CUSTOM_MODEL_NAME = 'model'
CONFIG_PATH = f'{MODEL_PATH}/{CUSTOM_MODEL_NAME}/pipeline.config'
CHECKPOINT_PATH = f'{MODEL_PATH}/{CUSTOM_MODEL_NAME}/checkpoint'

In [ ]:
!wget -O {PRETRAINED_MODEL_PATH}/{CUSTOM_MODEL_NAME}.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d1_coco17_tpu-32.tar.gz && tar -xf {PRETRAINED_MODEL_PATH}/{CUSTOM_MODEL_NAME}.tar.gz -C {PRETRAINED_MODEL_PATH} && mkdir {MODEL_PATH}/{CUSTOM_MODEL_NAME} || mv {PRETRAINED_MODEL_PATH}/{CUSTOM_MODEL}/pipeline.config {MODEL_PATH}/{CUSTOM_MODEL_NAME}

# 1. Create Label Map and TF Records

In [ ]:
labels = [{'name':'hoop', 'id':1}]

with open(ANNOTATION_PATH + '\label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write(f"\tname:\'{label['name']}\'\n")
        f.write(f"\tid:{label['id']}\n")
        f.write('}\n')

In [ ]:
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x {IMAGE_PATH + '/train'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/train.record'}
!python {SCRIPTS_PATH + '/generate_tfrecord.py'} -x{IMAGE_PATH + '/test'} -l {ANNOTATION_PATH + '/label_map.pbtxt'} -o {ANNOTATION_PATH + '/test.record'}